# 自動生成工具使用範例


## 匯入所需的套件


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## 建立客戶端

在此範例中，我們將使用 [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) 來存取 LLM。

`model` 被定義為 `gpt-4o-mini`。嘗試將模型更改為 GitHub Models 市場中提供的其他模型，看看不同的結果。

作為快速測試，我們將執行一個簡單的提示 - `法國的首都是什麼？`


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 定義函式

在這個範例中，我們將讓代理可以使用一個工具，該工具是一個包含可用度假目的地及其可用性的函式。

你可以將此情境想像成一個旅行代理可能擁有訪問旅行資料庫的權限，例如。

在閱讀這個範例時，請隨時嘗試定義新的函式和工具，讓代理可以調用它們。


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## 定義 Function Tool
為了讓代理能將 `vacation_destinations` 作為 `FunctionTool` 使用，我們需要將其定義為一個工具。

我們還會提供該工具的描述，這有助於代理識別該工具在與使用者請求的任務相關時的用途。


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## 定義代理

現在我們可以在以下程式碼中建立代理。我們定義了 `system_message`，用來向代理提供指導，幫助使用者尋找度假目的地。

我們還將 `reflect_on_tool_use` 參數設置為 true。這允許使用 LLM 接收工具調用的回應，並以自然語言發送回應。

你可以將該參數設置為 false，來觀察其中的差異。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## 執行代理

現在我們可以執行代理，初始的使用者訊息是要求安排一次東京旅行。

您可以更改這個城市目的地，來查看代理如何回應該城市的可用性。


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。我們致力於提供準確的翻譯，但請注意，自動翻譯可能包含錯誤或不準確之處。應以原始語言的文件作為權威來源。對於關鍵資訊，建議尋求專業人工翻譯。我們對因使用此翻譯而產生的任何誤解或錯誤解讀概不負責。
